In [74]:
from IPython.display import display
import pandas as pd

# Functions

#### Timestamp Adjustment

In [75]:


def convert_datetime(df, column_name, output_name):
    # Separando corretamente 'date_part' e 'time_part' na vírgula
    df[['date_part', 'time_part']] = df[column_name].str.split(',', expand=True)

    # Convertendo 'date_part' para numérico (parte da data no formato serial Excel)
    df['date_part'] = pd.to_numeric(df['date_part'], errors='coerce')

    # Convertendo a data de número serial Excel para datetime
    df['date'] = pd.to_datetime(df['date_part'], unit='D', origin='1899-12-30')

    # Reformatando 'time_part' para adicionar o ponto decimal correto
    # Convertendo '000226262' -> 0.000226262
    df['time_part'] = df['time_part'].apply(lambda x: float(x) / 1e9 if x else 0)

    # Convertendo 'time_part' diretamente em um timedelta usando a fração do dia
    df['time'] = pd.to_timedelta(df['time_part'], unit='D')

    # Somando 'date' e 'time' para obter o timestamp final
    df[output_name] = df['date'] + df['time']

    # Limpar as colunas intermediárias
    df = df.drop(columns=['date_part', 'time_part', 'time'])

    return df

#### Values Adjusted

In [76]:

def convert_currency_type(df, column_name):

    df[column_name] = df[column_name].str.replace(',', '').astype(float)

    return df

#### Currency Conversion

In [77]:
def convert_currency(df, column_list, exchange_rate_column):
    
    for column in column_list:
        columns_name = f"{column}_euro"

        print(columns_name)
        df[columns_name] = df[column] * df[exchange_rate_column]

    return df

# Adding Datasources

In [78]:
import pandas as pd

exchange_rate = pd.read_csv('Files/ExchangeRate.csv')
game = pd.read_csv('Files/Game.csv')
player = pd.read_csv('Files/Player.csv')
game_category = pd.read_csv('Files/GameCategory.csv')
game_provider = pd.read_csv('Files/GameProvider.csv')
payment_transaction = pd.read_csv('Files/PaymentTransaction.csv')
game_transaction = pd.read_csv('Files/GameTransaction.csv')


# General Type Conversion

In [79]:
exchange_rate['date'] = pd.to_datetime(exchange_rate['date'])


## Game Transaction

In [80]:
game_transaction = convert_datetime(game_transaction, 'transactionDateTime','transaction_timestamp')


#materializing joins to see data about games
game_transaction = game_transaction.join(exchange_rate.set_index(['currency','date']), on=['txCurrency','date'])
game_transaction = game_transaction.join(game.set_index(['ID']), on=['gameID'])
game_transaction = game_transaction.join(game_category.set_index(['ID']), on=['gameID'])

game_transaction['transaction_hour'] = game_transaction['transaction_timestamp'].dt.floor('H')



game_transaction = convert_currency_type(game_transaction, 'realAmount')
game_transaction = convert_currency_type(game_transaction, 'bonusAmount')


game_transaction = convert_currency(game_transaction, ['realAmount', 'bonusAmount'], 'baseRate')


game_transaction['wager_value'] = game_transaction.apply(
    lambda row: row['realAmount_euro'] if row['txType'] == "WAGER" else None,
    axis=1)

game_transaction['result_value'] = game_transaction.apply(
    lambda row: row['realAmount_euro'] if row['txType'] == "RESULT" else None,
    axis=1)

game_transaction['wager_transaction_hour'] = game_transaction.apply(
    lambda row: row['transaction_hour'] if row['txType'] == "WAGER" else None,
    axis=1)

game_transaction['result_transaction_hour'] = game_transaction.apply(
    lambda row: row['transaction_hour'] if row['txType'] == "RESULT" else None,
    axis=1)









C:\Users\theo.lourencao_ifood\AppData\Local\Temp\ipykernel_21488\361942343.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  game_transaction['transaction_hour'] = game_transaction['transaction_timestamp'].dt.floor('H')


realAmount_euro
bonusAmount_euro


In [81]:
group_by_list = ['BetID', 'gameID', 'Game Category','Game Name','Date', 'date', 'playerID']

# Agregando os dados
game_bets = (game_transaction.groupby(group_by_list, dropna=False)  # Manter NaN explícito se necessário
             .agg(
                 transactionDateTime=('transactionDateTime', 'first'),
                 transaction_timestamp=('transaction_timestamp', 'first'),
                 wager_transaction_hour=('wager_transaction_hour', 'first'),
                 result_transaction_hour=('result_transaction_hour', 'first'),
                 transaction_hour=('transaction_hour', 'first'),
                 realAmount=('realAmount', 'sum'),
                 bonusAmount=('bonusAmount', 'sum'),
                 realAmount_euro=('realAmount_euro', 'sum'),
                 bonusAmount_euro=('bonusAmount_euro', 'sum'),
                 wager_value=('wager_value', 'sum'),
                 result_value=('result_value', 'sum'),
                 txCurrency=('txCurrency', 'first'),
                 txType=('txType', lambda x: ','.join(x.unique())),
                 baseRate=('baseRate', 'mean'),
                 channelUID=('channelUID', 'first'),
                 GameProviderId=('GameProviderId', 'first'),
                 GameName=('Game Name', 'first')
             ).reset_index())


game_bets['is_positive'] = game_bets['wager_value']-game_bets['result_value'] < 0
game_bets['profit_margin'] = (game_bets['result_value']-game_bets['wager_value'])/game_bets['wager_value']

## Payment Transaction

In [82]:

payment_transaction_df = payment_transaction


payment_transaction_df = convert_datetime(payment_transaction_df, 'transactionDateTime','transaction_timestamp')

payment_transaction_df = payment_transaction_df.join(exchange_rate.set_index(['currency','date']), on=['txCurrency','date'])

payment_transaction_df['transaction_hour'] = payment_transaction_df['transaction_timestamp'].dt.floor('H')

payment_transaction_df = convert_currency(payment_transaction_df, ['amount'], 'baseRate')


payment_transaction_df['cancelled_withdrawal_amount'] = payment_transaction_df.apply(
    lambda row: row['amount_euro'] if row['txType'] == "CANCEL_WITHDRAWAL" else None,
    axis=1)

payment_transaction_df['deposit_amount'] = payment_transaction_df.apply(
    lambda row: row['amount_euro'] if row['txType'] == "DEPOSIT" else None,
    axis=1)

payment_transaction_df['withdrawal_amount'] = payment_transaction_df.apply(
    lambda row: row['amount_euro'] if row['txType'] == "WITHDRAWAL" else None,
    axis=1)

payment_transaction_df['pending_withdrawal_amount'] = payment_transaction_df.apply(
    lambda row: row['amount_euro'] if row['txType'] == "PENDING_WITHDRAWAL" else None,
    axis=1)

bet_id_to_exclude = '3392334408'


game_transaction = game_transaction[game_transaction['BetID'] != bet_id_to_exclude]



amount_euro


C:\Users\theo.lourencao_ifood\AppData\Local\Temp\ipykernel_21488\174516396.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  payment_transaction_df['transaction_hour'] = payment_transaction_df['transaction_timestamp'].dt.floor('H')


# Generating new DataFrames

### Hourly By Country

#### Table Preparation

In [83]:
group_by_columns = ['transaction_hour','country','playerID','Game Category']

In [84]:
# Sum of realAmount for WAGER and RESULT
wager_sum = game_transaction['wager_value'].sum()
result_sum = game_transaction['result_value'].sum()
# Create a new DataFrame with the results
df = pd.DataFrame({'Answer': [wager_sum, result_sum]}, index=['WAGER', 'RESULT'])

In [85]:
df

,Answer
WAGER,96662.728724
RESULT,256544.486202


##### Gaming Transactions

In [86]:
group_by_columns = ['transaction_hour','country','playerID','Game Category']

player_game_transaction = game_transaction.join(player.set_index('playerID'), on='playerID')


fill_dict = {
    'transaction_hour': 'Unknown',
    'country': 'Unknown',
    'playerID': 0000,
    'Game Category': 'Unknown',
}
player_game_transaction = player_game_transaction.fillna(fill_dict)


hourly_player_game_transaction = (player_game_transaction
                                  .groupby(group_by_columns)
                                  .agg(
                                      game_transaction_volume=('transaction_hour', 'size'),
                                      game_total_amount=('realAmount', 'sum'),
                                      sum_wager_value=('wager_value', 'sum'),
                                      sum_result_value=('result_value', 'sum'),
                                      count_wager_operations=('txType', lambda x: (x == 'WAGER').sum()),
                                      count_result_operations=('txType', lambda x: (x == 'RESULT').sum())
).reset_index())



##### Payment Transactions

In [87]:
payment_transaction_df = payment_transaction_df.join(player.set_index('playerID'), on='playerID')

In [88]:

payment_transaction_df['Game Category'] = 'N/A'

hourly_payment_transaction = (payment_transaction_df
                                  .groupby(group_by_columns)
                                  .agg(
                                      payment_transaction_volume=('transaction_hour', 'size'),
                                      payment_transaction_total_amount=('amount_euro', 'sum'),
                                      cancelled_withdrawal_amount=('cancelled_withdrawal_amount', 'sum'),
                                      pending_withdrawal_amount=('pending_withdrawal_amount', 'sum'),
                                      deposit_amount=('deposit_amount', 'sum'),
                                      withdrawal_amount=('withdrawal_amount', 'sum'),
                                      cancelled_withdrawal_count=('txType', lambda x: (x == 'CANCEL_WITHDRAWAL').sum()),
                                      withdrawal_count=('txType', lambda x: (x == 'WITHDRAWAL').sum()),
                                      deposit_count=('txType', lambda x: (x == 'DEPOSIT').sum())).reset_index())

#### Union & Materializing

In [89]:
hourly_metrics = pd.merge(
    hourly_payment_transaction, 
    hourly_player_game_transaction, 
    on=group_by_columns, 
    how='outer'
)

In [90]:
hourly_metrics['profit'] = hourly_metrics['sum_wager_value']-hourly_metrics['sum_result_value']



In [91]:
hourly_metrics['profit'] = hourly_metrics['sum_wager_value']-hourly_metrics['sum_result_value']


In [92]:
hourly_metrics.to_csv('Files/hourly_metrics.csv', index=False)

## Game Transaction Detailed

In [93]:
game_transaction_detailed = game_bets.join(player.set_index('playerID'), on='playerID')

game_transaction_detailed.info()

game_transaction_detailed.to_csv('Files/game_transaction_detailed.csv', index=False) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17870 entries, 0 to 17869
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   BetID                    17870 non-null  object        
 1   gameID                   17870 non-null  int64         
 2   Game Category            15808 non-null  object        
 3   Game Name                17870 non-null  object        
 4   Date                     17870 non-null  object        
 5   date                     17870 non-null  datetime64[ns]
 6   playerID                 17870 non-null  int64         
 7   transactionDateTime      17870 non-null  object        
 8   transaction_timestamp    17870 non-null  datetime64[ns]
 9   wager_transaction_hour   17870 non-null  datetime64[ns]
 10  result_transaction_hour  17870 non-null  datetime64[ns]
 11  transaction_hour         17870 non-null  datetime64[ns]
 12  realAmount               17870 n

## Player Profile and Metrics

In [97]:
import pandas as pd

# Supondo que df_game e df_payment já foram carregados como game_transaction_detailed e payment_transaction_df

def categorize_player_betting_profit(df):
    # Calculando o lucro total do jogador
    player_profit = df

    def classify_profit(profit):
        if profit > 0:
            return 'Profitable'
        else:
            return 'Unprofitable'
        

    player_profit['Profit Category'] = df['profit'].apply(classify_profit)
    return player_profit

def calculate_account_balance(df):
    df['account_balance'] = df['deposit_amount'].fillna(0) - df['wager_value'].fillna(0) + df['result_value'].fillna(0) - df['withdrawal_amount'].fillna(0)
    return df

# Apply the function to the game_transaction_detailed DataFrame




def categorize_withdrawal_status(df):
    """
    Adiciona uma coluna que categoriza se o jogador sacou mais do que depositou
    ou depositou mais do que sacou.
    """
    withdrawal_status = df['withdrawal_amount'] - df['deposit_amount']
    df['Withdrawal Status'] = pd.cut(
        withdrawal_status,
        bins=[-float('inf'), 0, float('inf')],
        labels=['Deposited More', 'Withdrew More']
    )
    return df

def categorize_playing_time(df):
    df['hour_of_day'] = df['transaction_hour'].dt.hour
    def classify_time_of_day(hour):
        if 6 <= hour < 12:
            return 'Morning'
        elif 12 <= hour < 18:
            return 'Afternoon'
        elif 18 <= hour < 24:
            return 'Night'
        else:
            return 'Early Morning'
    df['Time Category'] = df['hour_of_day'].apply(classify_time_of_day)
    most_frequent_time = df.groupby('playerID')['Time Category'].agg(lambda x: x.mode()[0])
    return most_frequent_time

def most_played_game(df):
    most_played = df.groupby('playerID')['Game Name'].agg(lambda x: x.mode()[0])
    return most_played

def most_used_channel(df):
    most_used = df.groupby('playerID')['channelUID'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown')
    return most_used

def categorize_deposit(df):
    player_avg_deposit = df.groupby('playerID')['deposit_amount'].mean().fillna(0)
    deposit_categories = pd.cut(player_avg_deposit, bins=[-float('inf'), 25, 50, 75, float('inf')], labels=['0-25', '26-50', '51-75', '>75'])
    return deposit_categories

def categorize_betting(df):
    player_avg_betting = df.groupby('playerID')['wager_value'].mean().fillna(0)
    betting_categories = pd.cut(player_avg_betting, bins=[-float('inf'), 25, 50, 75, float('inf')], labels=['0-25', '26-50', '51-75', '>75'])
    return betting_categories

def categorize_betting_volume(df):
    bet_volume = df.groupby('playerID')['wager_value'].count()
    bet_volume_categories = pd.cut(bet_volume, bins=[-float('inf'), 25, 50, 75, float('inf')], labels=['Low', 'Medium', 'High', 'Very High'])
    return bet_volume_categories

def check_pending_withdrawal(df):
    pending_withdrawal = df.groupby('playerID')['txType'].apply(lambda x: 'PENDING_WITHDRAWAL' in x.values).astype(int)
    return pending_withdrawal

def categorize_cash_in_hand(df):
    df['cash_in_hand'] = df['deposit_amount'].fillna(0) - df['withdrawal_amount'].fillna(0)
    cash_categories = pd.cut(df.groupby('playerID')['cash_in_hand'].sum(), bins=[-float('inf'), 0, 50, 200, float('inf')], labels=['Negative', 'Low', 'Medium', 'High'])
    return cash_categories

def main(game_df, payment_df):
    # Agrupando dados de game transactions por player
    game_grouped = game_df.groupby('playerID').agg(
        wager_value=('wager_value', 'sum'),
        realAmount=('realAmount', 'sum'),
        result_value=('result_value', 'sum'),
        Game_Name=('Game Name', 'first'),
        channelUID=('channelUID', 'first'),
        transaction_hour=('transaction_hour', 'first'),
        transactionDateTime=('transactionDateTime', 'first')
    )
    game_grouped['profit'] = game_grouped['wager_value'] - game_grouped['result_value']


    
    # Agrupando dados de pagamentos por player
    payment_grouped = payment_df.groupby('playerID').agg(
        deposit_amount=('deposit_amount', 'sum'),
        withdrawal_amount=('withdrawal_amount', 'sum'),
        pending_withdrawal_amount=('pending_withdrawal_amount', 'sum'),
        cancelled_withdrawal_amount=('cancelled_withdrawal_amount', 'sum'),
        amount_euro=('amount_euro', 'sum'),
        transaction_hour=('transaction_hour', 'first')
    )


    
    # Adicionando as métricas ao DataFrame final
    
    game_play_time = categorize_playing_time(game_df)
    most_played_game_result = most_played_game(game_df)
    most_used_device = most_used_channel(game_df)
    deposit_categories = categorize_deposit(payment_df)
    betting_categories = categorize_betting(game_df)
    betting_volume_categories = categorize_betting_volume(game_df)
    pending_withdrawal_status = check_pending_withdrawal(payment_df)
    cash_in_hand_categories = categorize_cash_in_hand(payment_df)

    # Combinando tudo
    final_result = categorize_player_betting_profit(game_grouped)
    final_result = final_result.join(game_play_time.rename('Most Frequent Time'))
    final_result = final_result.join(most_played_game_result.rename('Most Played Game'))
    final_result = final_result.join(most_used_device.rename('Most Used Device'))
    final_result = final_result.join(deposit_categories.rename('Deposit Category'))
    final_result = final_result.join(betting_categories.rename('Betting Category'))
    final_result = final_result.join(betting_volume_categories.rename('Betting Volume Category'))
    final_result = final_result.join(pending_withdrawal_status.rename('Pending Withdrawal Status'))
    final_result = final_result.join(cash_in_hand_categories.rename('Cash In Hand Category'))

    
    # Integrando dados de pagamento no DataFrame final
    final_result = final_result.join(payment_grouped[['deposit_amount', 'withdrawal_amount','pending_withdrawal_amount','cancelled_withdrawal_amount' ,'amount_euro']])
    

    final_result = calculate_account_balance(final_result)
    final_result = categorize_withdrawal_status(final_result)


    # Retorna o DataFrame final com as informações
    return final_result

# Exemplo de uso:



final_result = main(player_game_transaction, payment_transaction_df)

final_result.to_csv('Files/player_metrics.csv', index=True)

In [95]:

final_result

,wager_value,realAmount,result_value,Game_Name,channelUID,transaction_hour,transactionDateTime,profit,Profit Category,Most Frequent Time,...,Deposit Category,Betting Category,Betting Volume Category,Pending Withdrawal Status,Cash In Hand Category,deposit_amount,withdrawal_amount,amount_euro,account_balance,Withdrawal Status
playerID,,,,,,,,,,,,,,,,,,,,,
116004723,2362.000000,6228.0,3866.000000,MARLEEN,DESKTOP,2017-03-20 17:00:00,"42814,713134468",-1504.000000,Unprofitable,Afternoon,...,0-25,0-25,Very High,0,Low,35.000000,0.000000,35.000000,1.539000e+03,Deposited More
116008056,14.374802,172.0,3.805095,BILLYE,IPAD,2017-03-20 02:00:00,"42814,101246944",10.569707,Profitable,Early Morning,...,0-25,0-25,Low,0,Low,10.569707,0.000000,10.569707,1.332268e-15,Deposited More
116010104,130.007399,4016.0,294.472043,DONNIE,DESKTOP,2017-03-20 10:00:00,"42814,426976285",-164.464644,Unprofitable,Afternoon,...,0-25,0-25,Very High,0,Medium,63.418243,0.000000,63.418243,2.278829e+02,Deposited More
116015483,69.548674,1714.0,111.616108,MARGARETTA,IPHONE,2017-03-20 06:00:00,"42814,268643785",-42.067435,Unprofitable,Morning,...,0-25,0-25,Very High,0,Medium,63.418243,0.000000,63.418243,1.054857e+02,Deposited More
116018788,393.298806,22128.0,1945.566008,VANESA,None,2017-03-20 06:00:00,"42814,250562789",-1552.267202,Unprofitable,Morning,...,0-25,0-25,Very High,1,Negative,137.406194,443.927703,1025.261600,1.245746e+03,Withdrew More
116021116,1007.000000,4028.0,3021.000000,VANESA,None,2017-03-20 06:00:00,"42814,282196412",-2014.000000,Unprofitable,Morning,...,51-75,26-50,Medium,1,Negative,55.000000,880.000000,1815.000000,1.189000e+03,Withdrew More
116023410,87.728570,2559.0,182.750238,BLANCA,None,2017-03-20 08:00:00,"42814,353227708",-95.021668,Unprofitable,Afternoon,...,26-50,0-25,High,0,Medium,126.836487,0.000000,126.836487,2.218582e+02,Deposited More
116025440,4474.605148,5733.0,2125.063314,SO,IPHONE,2017-03-20 13:00:00,"42814,543641551",2349.541834,Profitable,Afternoon,...,0-25,26-50,Very High,0,Low,11.511719,0.000000,11.511719,-2.338030e+03,Deposited More
116025700,292.358102,14361.0,1225.557552,MARRY,IPHONE,2017-03-20 06:00:00,"42814,256663762",-933.199450,Unprofitable,Morning,...,26-50,0-25,Very High,1,High,581.333897,0.000000,687.030969,1.514533e+03,Deposited More
